In [ ]:
!pip install elevenlabs
!pip install openai
!pip install numpy

In [67]:
import requests
import json
import elevenlabs
from openai import OpenAI
import time
import numpy
import uuid

In [12]:
with open('../credentials.json', 'r') as file:
    cred = json.load(file)

elevenlabs.set_api_key(cred["elevenlabs_api_key"])
client = OpenAI(api_key=cred["openai_api_key"])

assistant_id = "asst_nv9sFR2LXHefrpwws4EUT6xU"
voice_id = "vDxT1hhO5CjgzUI7e2vF"

my_assistant = client.beta.assistants.retrieve(assistant_id)

new_thread = client.beta.threads.create()

In [64]:
import subprocess
import sys

print(sys.executable)
python_interpreter = sys.executable

def video_generation(audio_path, audio_id):
    os.makedirs(os.path.join('results', audio_id))

    audio_file = audio_path  # Can be changed dynamically
    source_image = "./SadTalker/example_img.jpg"
    result_dir = "./results/{audio_id}"
    # Construct the command string
    command = f"!python ./SadTalker/inference.py --driven_audio {audio_file} --source_image {source_image} --result_dir {result_dir}"
    get_ipython().system(command)
    
    '''
    !python ./SadTalker/inference.py \
      --driven_audio test.wav \
      --source_image ./SadTalker/example_img.jpg \
      --result_dir ./results/
    '''
    
    videos = glob.glob("./results/{audio_id}/*.mp4")
    clip = VideoFileClip(videos[0])
    clip = resize(clip, height=420)
    clip.ipython_display()

c:\Users\jonathancallahan\Projects\guidance\.venv\Scripts\python.exe


In [14]:
def new_question(question_text):
  message_request = client.beta.threads.messages.create(
      thread_id = new_thread.id,
      role = "user",
      content = question_text
  )

  run = client.beta.threads.runs.create(
    thread_id = message_request.thread_id,
    assistant_id = my_assistant.id
  )

  run = client.beta.threads.runs.retrieve(
    thread_id = message_request.thread_id,
    run_id = run.id
  )

  def wait_for_completed():
      while True:
          response = client.beta.threads.runs.retrieve(thread_id = new_thread.id, run_id = run.id)
          if response.status == "completed":
              return response
          time.sleep(1)

  run_return = wait_for_completed()

  messages = client.beta.threads.messages.list(
    thread_id = message_request.thread_id
  )

  audio = elevenlabs.generate(
      text = messages.data[0].content[0].text.value,
          voice = voice_id
  )

  elevenlabs.save(audio, "test.wav")
  video_generation("test.wav", uuid.uuid4())

  #elevenlabs.play(audio)
  

In [66]:
video_generation("test.wav")


Python interpreter in script: c:\Users\jonathancallahan\Projects\guidance\.venv\Scripts\python.exe
Python interpreter in script: c:\Users\jonathancallahan\Projects\guidance\.venv\Scripts\python.exe
./SadTalker\./checkpoints\epoch_20.pth
./SadTalker\./checkpoints\auido2pose_00140-model.pth
./SadTalker\./checkpoints\auido2exp_00300-model.pth
./SadTalker\./checkpoints\facevid2vid_00189-model.pth.tar
./SadTalker\./checkpoints\mapping_00229-model.pth.tar
The generated video is named example_img##test in ./results\2023_12_21_15.33.17



landmark Det:: 100%|██████████| 1/1 [00:16<00:00, 16.74s/it]

 3DMM Extraction In Video:: 100%|██████████| 1/1 [00:00<00:00,  1.23it/s]

Face Renderer:: 100%|██████████| 72/72 [00:40<00:00,  1.78it/s]
ffmpeg version 2023-11-15-git-78f55457c9-full_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --pkg-config=pkgconf --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enab

In [52]:
new_question("Hello")

Command failed
Traceback (most recent call last):
  File "./SadTalker/inference.py", line 9, in <module>
    from src.utils.preprocess import CropAndExtract
  File "C:\Users\jonathancallahan\Projects\guidance\SadTalker\src\utils\preprocess.py", line 4, in <module>
    import numpy as np
ModuleNotFoundError: No module named 'numpy'



In [ ]:
!python -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!python -m pip install moviepy
!python -m pip install imageio==2.4.1
!python -m pip install -r requirements.txt
!python -m pip install cmake==3.25.2
!python -m pip install boost==0.1
!python -m pip install dlib-bin
!python -m pip install basicsr==1.4.2
!python -m pip install facexlib==0.2.5
!python -m pip install kornia==0.6.10
!python -m pip install face-alignment==1.3.5 yacs==0.1.8
### install gpfgan for enhancer
!python -m pip install git+https://github.com/TencentARC/GFPGAN

In [3]:
import glob
import matplotlib.pyplot as plt
from PIL import Image
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

In [5]:
pip install face-alignment==1.3.4

  Using cached face_alignment-1.3.4-py2.py3-none-any.whl
  Attempting uninstall: face-alignment
    Found existing installation: face-alignment 1.3.5
    Uninstalling face-alignment-1.3.5:
      Successfully uninstalled face-alignment-1.3.5
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!python ./SadTalker/inference.py \
  --driven_audio ./SadTalker/examples/driven_audio/RD_Radio31_000.wav \
  --source_image ./SadTalker/example_img.jpg \
  --result_dir ./results

In [17]:
import os

# Path to the directory you want to scan
directory_path = 'results/2023_12_19_20.56.50/'

# List to hold the names of folders
file_names = []

# Walk through the directory
for entry in os.scandir(directory_path):
    
    file_names.append(entry.name)

# Print the folder names
print(file_names)

['example_img##RD_Radio31_000.mat', 'example_img##RD_Radio31_000.mp4', 'example_img##RD_Radio31_000.txt', 'first_frame_dir']


In [22]:
import os
print(os.getcwd())
%ls
directory_path = 'results/2023_12_19_20.56.50/'
file_name = 'example_img##RD_Radio31_000.mp4'

full_path = os.path.join(directory_path, file_name)

os.startfile(full_path)
'''
if os.path.exists(video_path):
    print("File exists")
    os.startfile(video_path)
else:
    print("File not found")
    '''

c:\Users\jonathancallahan\Projects\guidance
 Volume in drive C is Windows
 Volume Serial Number is C402-5B7C

 Directory of c:\Users\jonathancallahan\Projects\guidance

12/19/2023  09:19 PM    <DIR>          .
12/19/2023  09:19 PM    <DIR>          ..
11/17/2023  07:21 PM    <DIR>          .ipynb_checkpoints
11/26/2023  02:13 PM    <DIR>          .venv
12/19/2023  07:31 PM    <DIR>          checkpoints
11/26/2023  12:30 PM    <DIR>          content
11/17/2023  06:13 PM                10 README.md
12/19/2023  08:56 PM    <DIR>          results
12/19/2023  07:31 PM    <DIR>          SadTalker
12/19/2023  09:19 PM            10,914 text_to_speech.ipynb
11/17/2023  11:05 PM    <DIR>          video
               2 File(s)         10,924 bytes
               9 Dir(s)  522,661,634,048 bytes free


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'results/2023_12_19_20.56.50/example_img##RD_Radio31_000.mp4'